In [32]:
import cv2, os, numpy as np

In [33]:
target_directory = 'FullDataset' 
output_directory = 'TrainingImages'

In [34]:
os.makedirs(output_directory, exist_ok=True)

In [ ]:
import cv2
import numpy as np
import os

def transformar_para_quadrada(imagem, tamanho_final):
    # Obter as dimensões da imagem
    altura, largura = imagem.shape[:2]

    # Calcular o tamanho do quadrado
    lado_quadrado = max(altura, largura)

    # Criar uma nova imagem quadrada com fundo preto
    imagem_quadrada = np.zeros((lado_quadrado, lado_quadrado, 3), dtype=np.uint8)

    # Colocar a imagem original no centro da nova imagem quadrada
    y_offset = (lado_quadrado - altura) // 2
    x_offset = (lado_quadrado - largura) // 2
    
    # Converter a imagem em escala de cinza para 3 canais
    imagem_colorida = cv2.cvtColor(imagem, cv2.COLOR_GRAY2BGR)
    imagem_quadrada[y_offset:y_offset + altura, x_offset:x_offset + largura] = imagem_colorida

    # Redimensionar para o tamanho final desejado
    imagem_quadrada = cv2.resize(imagem_quadrada, (tamanho_final, tamanho_final), interpolation=cv2.INTER_CUBIC)

    return imagem_quadrada

for letter_folder in os.listdir(target_directory):
    letter_path = os.path.join(target_directory, letter_folder)
    
    if os.path.isdir(letter_path):
        for image_file in os.listdir(letter_path):
            image_path = os.path.join(letter_path, image_file)
            
            try:
                image = cv2.imread(image_path)
                if image is not None:
                    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    imagem_final = transformar_para_quadrada(gray_image, 512)
                    
                    largura_borda = 10  # ajuste conforme necessário
                    imagem_com_borda = cv2.copyMakeBorder(
                        imagem_final,
                        largura_borda,
                        largura_borda,
                        largura_borda,
                        largura_borda,
                        cv2.BORDER_CONSTANT,
                        value=[0, 0, 0]  # cor da borda (preto, por exemplo)
                    )

                    # Garantir que a imagem final não ultrapasse 512x512
                    imagem_com_borda = cv2.resize(imagem_com_borda, (512, 512), interpolation=cv2.INTER_CUBIC)
                                        
                    output_path = os.path.join(output_directory, letter_folder)
                    os.makedirs(output_path, exist_ok=True)
                    
                    gray_image_path = os.path.join(output_path, image_file)
                    cv2.imwrite(gray_image_path, imagem_com_borda)
       
            except Exception as e:
                print(f"Erro ao abrir a imagem {image_path} : {e}")
